## 11.7 Foody

Aplikacja będzie podobna do poprzedniej, tym razem wykorzystamy `api` https://www.themealdb.com/. Przygotujemy trzy fragmenty
- `FoodListFragment` - fragment wyświetlający listę wszystkich potraw - przefiltrujemy po kraju
- `FoodDetailFragment` - fragment wyświetlający dokładniejsze informacje o danej potrawie
- `FavoriteFragment` - fragment wyświtlający listęv ulubionych potraw

Wykorzystamy `BottomNavigation`, na którym umieścimy fragmenty listy potraw oraz listę ulubionych. Na fragment wyświetlający dokładniejsze infoprmacje będziemy mogli się przenieść z obu fragmentów - przekazany zostanie `id` potrawy. Do stworzenia listy ulubionych potraw wykorzystamy lokalną bazę `ROOM`.

W tej aplikacji posłużymy się pojedynczym `ViewModel` powiązanym z aktywnością hostującą.

<table><tr><td><img src="https://media0.giphy.com/media/lpzoI5Wm7JrPHlXdpd/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/5Zd75RkXZeDJNb46bF/giphy.gif" width="200" /></td></tr></table>

Dodajmy odpowiednie zależności

In [ ]:
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath "androidx.navigation:navigation-safe-args-gradle-plugin:2.5.2"
    }
}

In [ ]:
plugins {
    id 'com.android.application'
    id 'org.jetbrains.kotlin.android'
    id 'androidx.navigation.safeargs.kotlin'
    id 'kotlin-android'
    id 'kotlin-kapt'
}
...
dependencies {

    // ROOM
    implementation("androidx.room:room-ktx:2.4.3")
    kapt("androidx.room:room-compiler:2.4.3")

    // ViewModel
    implementation 'androidx.lifecycle:lifecycle-viewmodel-ktx:2.5.1'
    // LiveData
    implementation "androidx.lifecycle:lifecycle-livedata-ktx:2.5.1"

    // Fragment
    implementation "androidx.fragment:fragment-ktx:1.5.3"

    // Navigation
    implementation "androidx.navigation:navigation-fragment-ktx:2.5.2"
    implementation "androidx.navigation:navigation-ui-ktx:2.5.2"

    // Retrofit
    implementation 'com.squareup.retrofit2:retrofit:2.9.0'
    implementation 'com.squareup.retrofit2:converter-gson:2.9.0'

    // OkHttp
    implementation 'com.squareup.okhttp3:logging-interceptor:4.10.0'

    // Glide
    implementation 'com.github.bumptech.glide:glide:4.13.2'
    annotationProcessor 'com.github.bumptech.glide:compiler:4.13.2'
    ...
}

Dodajmy również odpowiednie upoważnienie do `AndroidManifest`

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

### **`Navigation`**

Rozpocznijmy od utworzenia fragmentów i dodania nawigacji

<img src="https://fv9-4.failiem.lv/down.php?i=bu3zkrzvs" width="400" />

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/foodListFragment">

    <fragment
        android:id="@+id/foodListFragment"
        android:name="pl.udu.uwr.pum.foody.ui.fragments.FoodListFragment"
        android:label="fragment_food_list"
        tools:layout="@layout/fragment_food_list" >
        <action
            android:id="@+id/action_foodListFragment_to_foodDetailFragment"
            app:destination="@id/foodDetailFragment" >
            <argument
                android:name="id"
                app:argType="string" />
        </action>
    </fragment>
    <fragment
        android:id="@+id/foodDetailFragment"
        android:name="pl.udu.uwr.pum.foody.ui.fragments.FoodDetailFragment"
        android:label="fragment_food_detail"
        tools:layout="@layout/fragment_food_detail" >
        <action
            android:id="@+id/action_foodDetailFragment_to_foodListFragment"
            app:destination="@id/foodListFragment" />
        <action
            android:id="@+id/action_foodDetailFragment_to_favoriteFragment"
            app:destination="@id/favoriteFragment" />
    </fragment>
    <fragment
        android:id="@+id/favoriteFragment"
        android:name="pl.udu.uwr.pum.foody.ui.fragments.FavoriteFragment"
        android:label="fragment_favorite"
        tools:layout="@layout/fragment_favorite" >
        <action
            android:id="@+id/action_favoriteFragment_to_foodDetailFragment"
            app:destination="@id/foodDetailFragment" >
            <argument
                android:name="id"
                app:argType="string" />
        </action>
    </fragment>
</navigation>

Następnie dodajmy `menu` dla `BottomNavigation`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/foodListFragment"
        android:icon="@drawable/ic_food"
        android:title="@string/food" />
    <item
        android:id="@id/favoriteFragment"
        android:icon="@drawable/ic_favorite"
        android:title="@string/favorite" />
</menu>

Dodajmy nawigację w klasie `MainActivity`

In [ ]:
class MainActivity : AppCompatActivity() {

    private val binding: ActivityMainBinding by lazy {
        ActivityMainBinding.inflate(layoutInflater)
    }

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    private val appBarConfiguration: AppBarConfiguration by lazy {
        AppBarConfiguration(setOf(R.id.foodListFragment, R.id.favoriteFragment))
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(binding.root)

        binding.bottomNavView.setupWithNavController(navController)
        setupActionBarWithNavController(navController, appBarConfiguration)
    }

    override fun onSupportNavigateUp(): Boolean {
        return navController.navigateUp(appBarConfiguration)
                || super.onSupportNavigateUp()
    }
}

Zwróćmy uwagę na `appBarConfiguration`, teraz mamy dwa fragmenty tzw. top level, z których chcemy przejść na inny fragment. Jako argument `AppBarCon figuration` nie podajemy `navController.graph` jak dotychczas, tylko zbiór wszystkich fragmentów które chcemy dodać do `BottomNavigation`

In [ ]:
private val appBarConfiguration: AppBarConfiguration by lazy {
    AppBarConfiguration(setOf(R.id.foodListFragment, R.id.favoriteFragment))
}

### **`Retrofit`**

Zmapujmy dane za pomocą pluginu `JSONToKotlin`, w efekcie dostaniemy kilkadziesiąt pól w klasie `Meal` - wybierzmy kilka

In [ ]:
data class MealResponse(
    val meals: List<Meal>
)

data class Meal(
    @PrimaryKey
    val idMeal: String,
    val strArea: String,
    val strCategory: String,
    val strInstructions: String,
    val strMeal: String,
    val strMealThumb: String,
    val strYoutube: String
)

Dodajmy interfejs `FoodApi` z dwiema metodami
- `getFood` - zwraca listę wszystkich potraw, jako `area` wybieramy Polskę
- `getFoodById` - zwraca potrawę o zadanych `id`

In [ ]:
interface FoodApi {
    @GET("api/json/v1/1/filter.php?a=Polish")
    suspend fun getFood() : Response<MealResponse>

    @GET("api/json/v1/1/lookup.php?")
    suspend fun getFoodById(@Query("i") id: String) : Response<MealResponse>
}

Dodajmy plik `Constants` w którym będziemy przechowywać nasz `baseUrl`

In [ ]:
const val baseUrl = "https://www.themealdb.com/"

Następnie utwórzmy instancję klasy `Retrofit`, również wykorzystamy `LoggingInterceptor`

In [ ]:
object RetrofitInstance {
    val api: FoodApi by lazy {
        val logging = HttpLoggingInterceptor()
        logging.setLevel(HttpLoggingInterceptor.Level.BODY)
        val client = OkHttpClient.Builder()
            .addInterceptor(logging)
            .build()
        Retrofit.Builder()
            .baseUrl(baseUrl)
            .addConverterFactory(GsonConverterFactory.create())
            .client(client)
            .build()
            .create(FoodApi::class.java)
    }
}

### **`ROOM`**

Będziemy chcieli dać użytkownikowi zapisanie ulubionych potraw w bazie danych. Rozpocznijmy od utworzenia `@Entity`, przejdźmy do klasy `Meal` i dodajmy odpowiednią adnotację.

In [ ]:
@Entity(tableName = "meal")
data class Meal(
    @PrimaryKey
    val idMeal: String,
    val strArea: String,
    val strCategory: String,
    val strInstructions: String,
    val strMeal: String,
    val strMealThumb: String,
    val strYoutube: String
)

Następnie utwórzmy interfejs `FoodDao` i dodajmy trzy metody pozwalające na wyciuągnięcie wszystkich elementów, dodanie i usunięcie elementu.

In [ ]:
@Dao
interface FoodDao {
    @Insert(onConflict = REPLACE)
    suspend fun insert(meal: Meal)

    @Delete
    suspend fun delete(meal: Meal)

    @Query("SELECT * FROM meal")
    fun getAllMeals() : LiveData<List<Meal>>
}

Zdefiniujmy również klasę reprezentującą bazę danych

In [ ]:
@Database(entities = [Meal::class], version = 1, exportSchema = false)
abstract class MealDatabase : RoomDatabase() {
    abstract fun foodDao(): FoodDao

    companion object{
        @Volatile private var INSTANCE: MealDatabase? = null

        fun getDatabase(context: Context): MealDatabase {
            return INSTANCE ?: synchronized(this){
                val instance = Room.databaseBuilder(
                    context.applicationContext,
                    MealDatabase::class.java,
                    "meal_database"
                ).build().also { INSTANCE = it }
                instance
            }
        }
    }
}

### **Szkielet `MVVM`**

Utwórzmy repozytorium z odpowiednimi metodami dostępowymi do naszego `api` oraz `ROOM`.

In [ ]:
class FoodRepository(private val foodDao: FoodDao) {
    suspend fun getFood() = RetrofitInstance.api.getFood()
    suspend fun getFoodById(id: String) = RetrofitInstance.api.getFoodById(id)

    val readAllData: LiveData<List<Meal>> = foodDao.getAllMeals()
    suspend fun insert(meal: Meal) = foodDao.insert(meal)
    suspend fun delete(meal: Meal) = foodDao.delete(meal)
}

Kolejną klasą będzie nazsz `FoodViewModel` - w tej aplikacji będziemy posiadać jeden `ViewModel` współdzielony przez wszystkie fragmenty. Ponieważ metoda `getDatabase` wymaga podania `Context`, musimy go przekazać w konstruktorze - wykorzystamy `Application` (najszerszy kontekst).

In [ ]:
class FoodViewModel(application: Application) : AndroidViewModel(application) {

Dodajmy repozytorium oraz dwa `MutableLiveData` w których dbierzemy wynik zapytań. Tutaj również skorzystamy z klasy `Resource` która pozwala w łatwiejszy sposób zarządzać stanem.

In [ ]:
private val repository: FoodRepository
private val _mealList: MutableLiveData<Resource<MealResponse>> = MutableLiveData()
private val _meal: MutableLiveData<Resource<MealResponse>> = MutableLiveData()

In [ ]:
sealed class Resource<T> (
    val data: T? = null,
    val message: String? = null
){
    class Success<T>(data: T) : Resource<T>(data)
    class Error<T>(message: String, data: T? = null) : Resource<T>(data, message)
    class Loading<T> : Resource<T>()
}

Dodajmy publiczne wartości wspomagające, zwracające `LiveData`

In [ ]:
val mealList: LiveData<Resource<MealResponse>>
    get() = _mealList

val meal: LiveData<Resource<MealResponse>>
    get() = _meal

Ostatnim polem jest `readAllData`, czyli `LiveData` przechowujący odpowiedź z lokalnej bazy

In [ ]:
val readAllData: LiveData<List<Meal>>

W bloku `init` inicjujemy `foodDao` repozytorium i `readAllData`

In [ ]:
init {
    val foodDao = MealDatabase.getDatabase(application).foodDao()
    repository = FoodRepository(foodDao)
    readAllData = repository.readAllData
}

Dodajmy funkcję pomocniczą `handleMealResponse`

In [ ]:
private fun handleMealResponse(response: Response<MealResponse>)
        : Resource<MealResponse> {
    if (response.isSuccessful)
        response.body()?.let { return Resource.Success(it) }
    return Resource.Error(response.message())
}

Nastrępnie zdefiniujmy funkcje zwracające listę potraw oraz potrawę po `id`

In [ ]:
fun getMealList() = viewModelScope.launch {
    _mealList.postValue(Resource.Loading())
    val response = repository.getFood()
    _mealList.postValue(handleMealResponse(response))
}

In [ ]:
fun getMealById(id: String) = viewModelScope.launch {
    _meal.postValue(Resource.Loading())
    val response = repository.getFoodById(id)
    _meal.postValue(handleMealResponse(response))
}

Ostatnim elementem będą funkcję dodania u usunięcia potrawy z lokalnej bazy

In [ ]:
fun insert(meal: Meal) = viewModelScope.launch {
    repository.insert(meal)
}

fun delete(meal: Meal) = viewModelScope.launch {
    repository.delete(meal)
}

### **`RecyclerView`**

Wykorzystamy `RecyclerView` dwa razy - dla listy wszystkich potraw, oraz listy ulubionych potraw. Zdefiniujmy więc layout pojedynczego elementu

In [ ]:
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:orientation="vertical"
    android:layout_marginTop="16dp">

    <TextView
        android:id="@+id/name"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:text="nazwa"
        android:textSize="24sp"
        android:layout_margin="8dp"/>

    <ImageView
        android:id="@+id/image"
        android:layout_width="match_parent"
        android:layout_gravity="center"
        android:layout_height="200dp"
        android:scaleType="centerCrop"
        android:layout_marginStart="25dp"
        android:layout_marginEnd="25dp"
        android:contentDescription="@string/food_image" />

</LinearLayout>

Dodajmy `Comparator`, który (podobnie jak layout) będzie dla obu wspólny

In [ ]:
class FoodComparator : DiffUtil.ItemCallback<Meal>() {
    override fun areItemsTheSame(oldItem: Meal, newItem: Meal): Boolean {
        return newItem.strMeal == oldItem.strMeal
    }

    override fun areContentsTheSame(oldItem: Meal, newItem: Meal): Boolean {
        return newItem.idMeal == oldItem.idMeal
    }
}

Następnie utwórzmy dwa `ViewHolder`

In [ ]:
class FoodViewHolder(private val binding: ListItemRvBinding)
    : RecyclerView.ViewHolder(binding.root){
        fun bind(item: Meal){
            binding.name.text = item.strMeal
            Glide.with(binding.root)
                .load(item.strMealThumb)
                .into(binding.image)
            binding.root.setOnClickListener {
                val action: NavDirections = FoodListFragmentDirections
                    .actionFoodListFragmentToFoodDetailFragment(
                        item.idMeal
                    )
                Navigation.findNavController(binding.root).navigate(action)
            }
        }
}

In [ ]:
class FavoriteViewHolder(private val binding: ListItemRvBinding)
    : RecyclerView.ViewHolder(binding.root){
        fun bind(item: Meal){
            binding.name.text = item.strMeal
            Glide.with(binding.root)
                .load(item.strMealThumb)
                .into(binding.image)
            binding.root.setOnClickListener {
                val action: NavDirections = FavoriteFragmentDirections
                    .actionFavoriteFragmentToFoodDetailFragment(
                        item.idMeal
                    )
                Navigation.findNavController(binding.root).navigate(action)
            }
        }
}

Wykorzystujemy `ImageView`, grafikę którą pobieramy wstawiamy za pomocą biblioteki `Glide`. Dodajmy również dwa adaptery.

In [ ]:
class FoodAdapter(itemComparator: FoodComparator)
    : ListAdapter<Meal, FoodViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): FoodViewHolder {
        return FoodViewHolder(
            ListItemRvBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }

    override fun onBindViewHolder(holder: FoodViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }
}

In [ ]:
class FavoriteAdapter(itemComparator: FoodComparator) 
    : ListAdapter<Meal, FavoriteViewHolder>(itemComparator) {
    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): FavoriteViewHolder {
        return FavoriteViewHolder(
            ListItemRvBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }

    override fun onBindViewHolder(holder: FavoriteViewHolder, position: Int) {
        val item = getItem(position)
        holder.bind(item)
    }

    fun getItemAt(position: Int): Meal{
        return getItem(position)
    }
}

### **`FoodListFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/foodRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ProgressBar
        android:id="@+id/progressBar"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_gravity="center"
        android:layout_marginTop="36dp"
        android:background="@android:color/transparent"
        android:visibility="invisible"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Utworzmy `ViewModel` - tym razem wykorzystamy `activityViewModel`.

In [ ]:
private val foodViewModel: FoodViewModel by activityViewModels()

Dodajmy funkcje pomocnicze

In [ ]:
private fun hideProgressBar(){
    binding.progressBar.visibility = View.INVISIBLE
}

private fun showProgressBar(){
    binding.progressBar.visibility = View.VISIBLE
}

private fun setupRecyclerView(foodAdapter: FoodAdapter) {
    binding.foodRV.apply {
        adapter = foodAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }
}

Ostatnią funkcją będzie `observeFood`

In [ ]:
private fun observeFood(foodAdapter: FoodAdapter) {
    foodViewModel.mealList.observe(viewLifecycleOwner) { response ->
        when (response) {
            is Resource.Success -> {
                hideProgressBar()
                response.data?.let { res ->
                    foodAdapter.submitList(res.meals)
                }
            }
            is Resource.Error -> {
                hideProgressBar()
                response.message?.let { Log.e(TAG, "Error occurred: $it") }
            }
            is Resource.Loading -> showProgressBar()
        }
    }
}

W `onViewCreated` wywołuję wcześniej zdefiniowane funkcje

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)

    foodViewModel.getMealList()

    val adapter = FoodAdapter(FoodComparator())
    setupRecyclerView(adapter)

    observeFood(adapter)
}

### **`FoodDetailFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<ScrollView xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <RelativeLayout
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        tools:context=".DetailActivity">

        <ProgressBar
            android:id="@+id/progressBar"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignParentStart="true"
            android:layout_alignParentTop="true"
            android:layout_alignParentEnd="true"
            android:layout_gravity="center"
            android:layout_marginStart="181dp"
            android:layout_marginTop="35dp"
            android:layout_marginEnd="182dp"
            android:background="@android:color/transparent"
            android:visibility="invisible" />

        <ImageView
            android:id="@+id/foodImage"
            android:layout_width="match_parent"
            android:layout_height="wrap_content"
            android:adjustViewBounds="true"
            android:contentDescription="@string/food_image" />

        <TextView
            android:id="@+id/title"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:padding="8dp"
            android:text="@string/title"
            android:theme="@style/ThemeOverlay.AppCompat.Dark" />

        <TextView
            android:id="@+id/category"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/foodImage"
            android:padding="8dp"
            android:text="@string/category"
            android:textColor="?android:textColorSecondary" />

        <TextView
            android:id="@+id/instructions"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_below="@id/category"
            android:padding="8dp"
            android:text="@string/instructions" />

        <com.google.android.material.floatingactionbutton.FloatingActionButton
            android:id="@+id/favoriteButton"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_alignBottom="@id/foodImage"
            android:layout_alignParentEnd="true"
            android:layout_marginEnd="8dp"
            android:layout_marginTop="16dp"
            android:contentDescription="@string/favorite_button"
            android:src="@drawable/ic_favorite_border" />
    </RelativeLayout>
</ScrollView>

Utwórzmy `ViewModel` i odbierzmy argument

In [ ]:
private val foodViewModel: FoodViewModel by activityViewModels()

private val id: String? by lazy { requireArguments().getString("id") }

W metodzie `onCreate` wywołajmy `getMealById` 

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentFoodDetailBinding.inflate(layoutInflater, container, false)
    foodViewModel.getMealById(id!!)
    return binding.root
}

Dodajmy funkcje pomocnicze

In [ ]:
private fun inflate(item: Meal) {
    Glide.with(this)
        .load(item.strMealThumb)
        .into(binding.foodImage)
    binding.category.text = item.strCategory
    binding.title.text = item.strMeal
    binding.instructions.text = item.strInstructions
}

private fun hideProgressBar(){
    binding.progressBar.visibility = View.INVISIBLE
}

private fun showProgressBar(){
    binding.progressBar.visibility = View.VISIBLE
}

Dodajmy funkcję `observeMeal`

In [ ]:
private fun observeMeal() {
    foodViewModel.meal.observe(viewLifecycleOwner) { response ->
        when (response) {
            is Resource.Success -> {
                hideProgressBar()
                response.data?.let { res ->
                    val item = res.meals.first()
                    inflate(item)
                    binding.favoriteButton.setOnClickListener {
                        foodViewModel.insert(item)
                    }
                }
            }
            is Resource.Error -> {
                hideProgressBar()
                response.message?.let { Log.e(TAG, "Error occurred: $it") }
            }
            is Resource.Loading -> showProgressBar()
        }
    }
}

Ponieważ dostaję odpowiedź jako listę, tutaj będzie tob lista jednoelementowa, więc wykorzystuję funkcję `first`

In [ ]:
val item = res.meals.first()

Następnie wywołuję funkcję `inflate` zdefiniowaną wcześniej

In [ ]:
inflate(item)

Dodaję również obsługę `onClick` przycisku dodającego potrawę do ulubionych

In [ ]:
binding.favoriteButton.setOnClickListener {
    foodViewModel.insert(item)
}

### **`FavoriteFragment`**

Rozpocznijmy od layoutu

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/favoriteRV"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_margin="8dp"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Dodajmy `ViewModel`

In [ ]:
private val foodViewModel: FoodViewModel by activityViewModels()

Dodajmy dwie funkcje pomocnicze

In [ ]:
private fun setupRecyclerView(favoriteAdapter: FavoriteAdapter) {
    binding.favoriteRV.apply {
        adapter = favoriteAdapter
        layoutManager = LinearLayoutManager(requireContext())
    }
}

private fun swipeToDelete(adapter: FavoriteAdapter) {
    ItemTouchHelper(object : ItemTouchHelper.SimpleCallback(
        0,
        ItemTouchHelper.RIGHT or ItemTouchHelper.LEFT
    ) {
        override fun onMove(
            recyclerView: RecyclerView,
            viewHolder: RecyclerView.ViewHolder,
            target: RecyclerView.ViewHolder
        ): Boolean {
            return false
        }

        override fun onSwiped(viewHolder: RecyclerView.ViewHolder, direction: Int) {
            foodViewModel.delete(adapter.getItemAt(viewHolder.adapterPosition))
        }
    }).attachToRecyclerView(binding.favoriteRV)
}

W metodzie `onViewCreated` wywołujemy wcześniej zdefiniowane metody

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)

    val adapter = FavoriteAdapter(FoodComparator())
    setupRecyclerView(adapter)
    foodViewModel.readAllData.observe(viewLifecycleOwner, adapter::submitList)
    swipeToDelete(adapter)
}

Możemy przetestować aplikację

<table><tr><td><img src="https://media0.giphy.com/media/lpzoI5Wm7JrPHlXdpd/giphy.gif" width="150" /></td><td><img src="https://media2.giphy.com/media/5Zd75RkXZeDJNb46bF/giphy.gif" width="150" /></td></tr></table>